# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Thu 2023-01-26 10:33:28.756129 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
Thu 2023-01-26 10:33:28.756696 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
Thu 2023-01-26 10:33:28.756736 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Thu 2023-01-26 10:33:28.758984 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Thu 2023-01-26 10:33:28.781762 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f08f41be9d0>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
{"c": client_tiface, "s": server_tiface}

{'c': <__main__.LoggingTransportInterface at 0x7f08f407a1a0>,
 's': <__main__.LoggingTransportInterface at 0x7f08f407a110>}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7f08f4067ba0>,
 's': <FilterInterface at 0x7f08f40d8d50>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(server)0: got transport notifications: []
LTI(client)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server as ping endpoint

In [10]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013)] }

In [11]:
await client_fiface.command_response(filter_cmd)

Thu 2023-01-26 10:33:34.853690 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013)] }


Accepted

LTI(client)57 skipped 56: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 16213713991968370759 } }] }
Thu 2023-01-26 10:33:36.305327 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 16213713991968370759 } }] }
LTI(client)58: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-26 10:33:36.305558 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 16213713991968370759 } }
Thu 2023-01-26 10:33:36.306659 [TRACE] - [F<-T,R] Command Accepted
Thu 2023-01-26 10:33:36.330596 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:47344), Took packet GetStatus { ping: PingPong { nonce: 16213713991968370759 } }
LT

# XXX BELOW

In [12]:
server_notifications = server_fiface.get_notifications()
server_notifications

[]

Thu 2023-01-26 10:33:38.825640 [WARN ] - [F] error sending KeepAlive during idle endpoint (Endpoint(127.0.0.1:62013)): Filter does not contain an entry for the endpoint: Endpoint(127.0.0.1:62013)
Thu 2023-01-26 10:33:41.318611 [INFO ] - [F<-T,N] Endpoint Endpoint(127.0.0.1:47344) timed-out. Dropping.
LTI(server)290 skipped 289: received transport_cmd DropEndpoint { endpoint: Endpoint(127.0.0.1:47344) }
Thu 2023-01-26 10:33:41.320105 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:47344) }
LTI(server)291: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-26 10:33:41.320898 [TRACE] - [F<-T,R] Command Accepted
LTI(server)292: got transport notifications: []


In [ ]:
await server_fiface.command_response(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

In [ ]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

In [ ]:
new_response_code.response_code

In [ ]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)